In [1]:
import gradio as gr
import numpy as np
import string
import os
import torch
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.cross_encoder import CrossEncoder
from rank_bm25 import BM25Okapi

from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.document_loaders import PyPDFLoader, PDFMinerLoader
from langchain.vectorstores import FAISS

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import re
import pandas as pd

In [2]:
loader = PDFMinerLoader("/home/skmlab/data/quangminh/ProjectII/document/luatgiaothong.pdf")
text = loader.load()

In [3]:
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=100)
#text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
docs = text_splitter.split_documents(text)

Created a chunk of size 523, which is longer than the specified 512


In [4]:
len(docs)

254

In [5]:
def get_ref(chunk):
    chunk = chunk.split("\n\n")
    doc_ref = []
    for doc in chunk:
        rules = re.findall('Điều \d+\..*', doc)
        if rules != []:
            doc_ref.append(rules)
            
    return doc_ref[0][0]

In [6]:
chunks = []
for doc in docs:
    chunks.append(doc.page_content)
len(chunks)

254

In [7]:
ref = []
for chunk in chunks:
    try:
        ref.append(get_ref(chunk))
    except:
        ref.append(ref[-1])

In [8]:
len(ref)

254

In [9]:
ref[-1]

'Điều 88. Hiệu lực thi hành '

In [10]:
data = {'ref': ref, 'chunk': chunks}
df = pd.DataFrame(data)
df.to_csv('/home/skmlab/data/quangminh/ProjectII/db.csv')
#print(df)

In [11]:
ref[15]

'Điều 4. Nguyên tắc hoạt động giao thông đường bộ'

In [12]:
chunks[0]

'QUỐC HỘI \nLuật số: 23/2008/QH12 \n\nCỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM \nĐộc lập - Tự do - Hạnh phúc \n\nLUẬT \n\nGIAO THÔNG ĐƯỜNG BỘ  \n\nCăn  cứ  Hiến  pháp  nước  Cộng  hòa  xã  hội  chủ  nghĩa  Việt  Nam  năm  1992  đã \n\nđược sửa đổi, bổ sung một số điều theo Nghị quyết số 51/2001/QH10; \n\nQuốc hội ban hành Luật giao thông đường bộ. \n\nCHƯƠNG I \n\nNHỮNG QUY ĐỊNH CHUNG \n\nĐiều 1. Phạm vi điều chỉnh'

In [13]:
chunks[1]

'CHƯƠNG I \n\nNHỮNG QUY ĐỊNH CHUNG \n\nĐiều 1. Phạm vi điều chỉnh \n\nLuật này quy định về quy tắc giao thông đường bộ; kết cấu hạ tầng giao thông \nđường bộ; phương tiện và người tham gia giao thông đường bộ; vận tải đường bộ và \nquản lý nhà nước về giao thông đường bộ. \n\nĐiều 2. Đối tượng áp dụng \n\nLuật này áp dụng đối với tổ chức, cá nhân liên quan đến giao thông đường bộ \n\ntrên lãnh thổ nước Cộng hòa xã hội chủ nghĩa Việt Nam. \n\nĐiều 3. Giải thích từ ngữ \n\nTrong Luật này, các từ ngữ dưới đây được hiểu như sau:'